Primero definimos una encriptacion del tipo repeated pad 

In [113]:
def encript(message, key, n_alphabet):
    # Asumimos que tanto el mensaje como la llave vienen codificadas de una manera numerica
    enc_message = [] # Retornaremos el mensaje encriptado como una lista
    for i in range(len(message)):
        enc_val = message[i] + key[i % len(key)]
        enc_message.append(enc_val % n_alphabet)
    return enc_message

def decript(enc_msg, key, n_alfabet):
    message = [] # Retornaremos el mensaje encriptado como una lista
    for i in range(len(enc_msg)):
        dec_val = enc_msg[i] - key[i % len(key)]
        message.append(dec_val % n_alfabet)
    return message

In [5]:
e = encript([0,2,4,3,6,7], [1,2,3], 10) # Usamos alfabeto solo de 0-9
print(e)
d = decript(e, [1,2,3], 10)
print(d)

[1, 4, 7, 4, 8, 0]
[0, 2, 4, 3, 6, 7]


Como se puede chequear la encriptacion funciona como corresponde: 

0 + 1 mod 10 = 1

2 + 2 mod 10 = 4

...

7 + 3 mod 10 = 0

Y de la misma forma la decriptación devuelve el mensaje original.

Definimos ahora métodos de utilidad para convertir un string cualquiera a una lista de numeros

In [36]:
def codify_message(message, frequencies):
    # Esta funcion recibe el mismo arreglo de frequencias que break_rp
    mapping_dict = {}
    revert_dict = {}
    i = 0
    for key in sorted(frequencies.keys()):
        mapping_dict[key] = i
        revert_dict[i] = key
        i += 1
    
    new_message = []
    
    for j in message:
        new_message.append(mapping_dict[j])

    return new_message, mapping_dict, revert_dict

In [37]:
c, mapping_dict, revert_dict = codify_message('LOLOLXDXD'*10, {'L': 0.25, 'O': 0.25, 'X': 0.25, 'D': 0.25})
print(c)
original = "".join(list(map(lambda x: revert_dict[x], c)))
print(original)

[1, 2, 1, 2, 1, 3, 0, 3, 0, 1, 2, 1, 2, 1, 3, 0, 3, 0, 1, 2, 1, 2, 1, 3, 0, 3, 0, 1, 2, 1, 2, 1, 3, 0, 3, 0, 1, 2, 1, 2, 1, 3, 0, 3, 0, 1, 2, 1, 2, 1, 3, 0, 3, 0, 1, 2, 1, 2, 1, 3, 0, 3, 0, 1, 2, 1, 2, 1, 3, 0, 3, 0, 1, 2, 1, 2, 1, 3, 0, 3, 0, 1, 2, 1, 2, 1, 3, 0, 3, 0]
LOLOLXDXDLOLOLXDXDLOLOLXDXDLOLOLXDXDLOLOLXDXDLOLOLXDXDLOLOLXDXDLOLOLXDXDLOLOLXDXDLOLOLXDXD


Ahora ya que podemos encriptar mensajes arbitrarios, procedemos a romper la cosa.

Primero definimos una funcion que nos ayudara a estimar el mejor largo de la llave. Básicamente lo que hacemos es, para todos los largos posibles de llave, buscar un conjunto que debiese estar cifrado por la misma llave. Luego, ordenamos la lista de las frecuencias y vamos comparando con las obtenidas en el conjunto. Si tienen sentido, la distancia debiera ser muy baja 

In [103]:
def get_key_len(array, frequencies):
    max_key_length = len(array) //50
    best = None
    dist = 100000
    for i in range(max_key_length):
        sub_msg = array[::i+1]
        counts = sorted([sub_msg.count(val) / len(sub_msg) for val in sorted(frequencies.keys())])
        dist_to_max = 0
        ground_truth = sorted(frequencies.values())
        for j in range(len(counts)):
            dist_to_max += abs(counts[j] - ground_truth[j])
        #print(i, dist_to_max)   
        if dist_to_max < dist:
            dist = dist_to_max
            best = i+1
    return best

In [47]:
get_key_len(c, {'L': 0.25, 'O': 0.25, 'X': 0.25, 'D': 0.25}, mapping_dict)

[20, 30, 20, 20] 30
0.3333333333333333


Teniendo eso es trivial hacer el ejercicio. Utilizando el acercamiento visto en clases, vamos comparando los conjuntos de letras que se encriptan con lo mismo y vemos, utilizando la funcion de distancia, si tiene sentido o no. Se elige la letra que mejor se ajuste y listo

In [106]:
def break_rp(
    ciphertext,
    frequencies,
    distance,
) -> str:

    """
    Arguments :
    ciphertext : An abritrary string representing the
    encrypted version of a plaintext .
    frequencies : A dictionary representing a character
    frequency over the alphabet .
    distance : A function indicating how distant is a string
    from following a character frequency
    Returns :
    key : A guess of the key used to encrypt the ciphertext , assuming
    that the plaintext message was written in a language in which
    letters distribute according to frequencies .
    """
    # Transformamos nuestro string a una lista
    codified_text, map_dict, rev_dict = codify_message(ciphertext, frequencies)

    # Primero nos gustaria saber que tan larga es la llave.
    key_len = get_key_len(ciphertext, frequencies)
    alphabet = sorted(frequencies.keys())
    print(key_len)
    key = ""
    for i in range(key_len):
        dist = 10000
        letter = None
        for j in alphabet:
            # Asumimos que j es la primera pos de la llave
            c_j = map_dict[j]
            # Obtenemos las letras en donde debiese usarse dicha llave
            segment = codified_text[i::key_len]
            # Decodificamos el mensaje
            dec = decript(segment, [c_j], len(alphabet))
            # mapeamos el mensaje a un string
            msg = "".join(map(lambda x: rev_dict[x], dec))
            # Utilizando la funcion de distancia comparamos
            curr_dist = distance(msg, frequencies)
            if curr_dist < dist:
                dist = curr_dist
                letter = j
        key += letter
    print('Predigo que la llave es:', key)
    return key



In [67]:
def abs_distance ( string , frequencies) -> float :
    """
    Arguments :
    string : An abritrary string
    frequencies : A dictionary representing a character frequency
    Returns :
    distance : How distant is the string from the character frequency
    """
    return sum([
    abs ( frequencies [c] - string . count ( c ) / len ( string ) )
    for c in frequencies
    ])


In [57]:
# Creamos una utilidad para generar una palabra random con cierta distribucion
import random
def createWord(n, frequencies):
    return "".join(random.choices(list(frequencies.keys()), weights=list(frequencies.values()), k=n))

In [112]:
f = {'L': 0.2, 'O': 0.06, 'X': 0.04, 'D': 0.7}
w = createWord(1000, f)
print('Mensaje normal:',w)
k = 'LOLXDO'
m, mapdict, revdict = codify_message(w, f)
k_c = []
for j in k:
    k_c.append(mapdict[j])
c = encript(m, k_c, len(f))
c_final = "".join(map(lambda x: revdict[x], c))
print('Mensaje Cifrado',c_final)
break_rp(c_final, f, abs_distance)



Mensaje normal: LDDDDODLDDDDLDODDDDDDLLLDODDDDDDDDDDDDDLDDDDDODDLLDDDDDDDDDDDDDDDDLDDLODDDDDDLDLDXLDDLDLLLDLDODDDDLXDLDDDDDLDLLDDDDDLDODLDODDDLLDDDDDDODXLDDDDODDDDDDODDLDDDODDDDDDODLDDDLDDDLDDDLDXDODDDDDDLLLDDDDODLDDDLDDLDLDLDLDDLDODLDDLLDDDODLDDLDLDDDDDLODDDXDLDDDLDDDLXLLXLDXDDLLDDDLDLDXDDDDDODDLDDDDLLLLODODLXDLDDDDDLLDODLODDLDDDLDDDDDDDDDLDDDDLDLDDDDDODDDDLLDDLODDDDLDDDDDDDDLLDDDDDXDLDDDXDDDDDDDDDLDDDOLDDDDDDDDDLXDLDDDLDDDODDDDDDDDDDXLLDLDDDDDDDDDDOLLDDDDLLDDDDDXLLDDDDDDDOLDXLODDLDDDDDDLDLXDDDLDDLDDXODDOLDDLDDLDDDDOLDDDDLDDDDDDLDXDLDDDDDDDDDDDDDLODDLLDLDDDDDDDLDDLDDDDDDDDDDDDDXDXDDLDLOLDLDDLDLLDDDDDDDOLXDDLOLLDDXDLDDDDDLDLDDOLDODDLDDLDLLDDDDDDOODDDDDODDDDDLDDXDDDDDDLDDDXDDDLDDDDLLDDDDLDDDXDDDDDDDDLODLDDDXDDLDDLDDLDLLODDDODDDDDXDDDDLDXLDDDDDDLDDDDDLDDDLDLDDDDDDLLDDLDDLDDDDDDXXDLDLDLDDDDDDLDDODODDLDDDDLDDDDDDLDLDLLDDODDDODDDDDDDDDDDDLDDDDDXDDDLDDLDDDDDLDDDDLDDLDDOOLLDLDXDDDDLLDLDDLDDDLODDDLXDDDDDOXLDDDDXDLXDDDLDDLDLLDDLXDDDLLLLXDDOOLXDDDLLDLDLDDDDDDDDDDDDDDDDDDXDLLDDDDDDDDDDDLLDDLDDDDD

'LOLXDO'